In [16]:
import os
import re
import math
import json
import random
from dotenv import load_dotenv
from huggingface_hub import login
import matplotlib.pyplot as plt
import numpy as np
import pickle
from collections import Counter
from openai import OpenAI
import httpx

In [ ]:
# CONCLUSIONS:
#
# - the more epochs, the better for training accuracy BUT there is a limit, if I put more than certain number it fails, I saw recommended is from 3-5 epochs
# - the more data, the better. openAI recommends 50-100 examples aprox. I initially tried with 6-8 rows and the result was really bad, then I tried with a 170 rows dataset and it worked better, at least answering cities and degrees
#   but it still needs to improve accuracy with the degrees because they don't match the 100% of the times
# - the answers improved A LOT changing the content of the training data from: { role-system, role-assistant } to: { role-system, role-user (question), role-assistant }
# - there is validation file parameter, it could be interesting to know how is the correct format to use it


In [17]:
model_name = "gpt-4o-mini"

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

openai = OpenAI(http_client=httpx.Client(verify=False))

In [22]:
test_prompts = [
    "what is the temperature in Tegucigalpa today?",
    "what is the temperature in Yamoussoukro today?",
    "what is the temperature in Rabat today?",
    "what is the temperature in Wellington today?",
    "what is the temperature in New York today?"
]

def ask_for_test_prompts(show_model=False):

    if show_model:
        print(f"using {model_name} model\n")
        
    for q in test_prompts:
        print(f"question: {q}\n")
        a = gpt_fine_tuned(q,show_model)
        print(f"  -answer: {a}\n")

In [23]:
def gpt_fine_tuned(user_prompt, show_model=False):

    system_message = "you are an assistant, just provide quick answers"

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt}
    ]
    response = openai.chat.completions.create(
        model=model_name, 
        messages=messages,
        seed=42,
        max_tokens=50,
        temperature=0.1
    )
    reply = response.choices[0].message.content
    return reply



In [24]:
# TEST BEFORE FINE TUNING

ask_for_test_prompts(True)

using gpt-4o-mini model

question: what is the temperature in Tegucigalpa today?

  -answer: I don't have real-time data access to provide current temperatures. Please check a weather website or app for the latest information.

question: what is the temperature in Yamoussoukro today?

  -answer: I don't have real-time data access to provide current temperatures. Please check a weather website or app for the latest information.

question: what is the temperature in Rabat today?

  -answer: I don't have real-time data access to provide current temperatures. Please check a weather website or app for the latest information.

question: what is the temperature in Wellington today?

  -answer: I don't have real-time data access to provide current temperatures. Please check a weather website or app for the latest information.

question: what is the temperature in New York today?

  -answer: I don't have real-time data access to provide current temperatures. Please check a weather website or ap

In [25]:
# load fine tuning data 

with open("./resources/fine_tuning_info.jsonl", "rb") as f:
    train_file = openai.files.create(file=f, purpose="fine-tune")

with open("./resources/fine_tuning_validation.jsonl", "rb") as f:
    validation_file = openai.files.create(file=f, purpose="fine-tune")

In [7]:
# run fine tuning

openai.fine_tuning.jobs.create(
    training_file=train_file.id,    
    model="gpt-4o-mini-2024-07-18",
    seed=42,
    hyperparameters={"n_epochs": 10},
    suffix="fine-tuned"
)

FineTuningJob(id='ftjob-1rodriZljdy3TddfP2tCD1gF', created_at=1747401764, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs=10), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-l5pA9yLCUC9WzyRoifiw8iGF', result_files=[], seed=42, status='validating_files', trained_tokens=None, training_file='file-C1iyTXfVQ6mrKdnidqriq3', validation_file=None, estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs=10)), type='supervised'), user_provided_suffix='fine-tuned', metadata=None, usage_metrics=None, shared_with_openai=False, eval_id=None)

In [8]:
openai.fine_tuning.jobs.list(limit=1)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-1rodriZljdy3TddfP2tCD1gF', created_at=1747401764, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs=10), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-l5pA9yLCUC9WzyRoifiw8iGF', result_files=[], seed=42, status='validating_files', trained_tokens=None, training_file='file-C1iyTXfVQ6mrKdnidqriq3', validation_file=None, estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs=10)), type='supervised'), user_provided_suffix='fine-tuned', metadata=None, usage_metrics=None, shared_with_openai=False, eval_id=None)], has_more=True, object='list')

In [26]:
job_id = openai.fine_tuning.jobs.list(limit=1).data[0].id

print(job_id)

ftjob-1rodriZljdy3TddfP2tCD1gF


In [10]:
openai.fine_tuning.jobs.retrieve(job_id)

FineTuningJob(id='ftjob-1rodriZljdy3TddfP2tCD1gF', created_at=1747401764, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs=10), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-l5pA9yLCUC9WzyRoifiw8iGF', result_files=[], seed=42, status='validating_files', trained_tokens=None, training_file='file-C1iyTXfVQ6mrKdnidqriq3', validation_file=None, estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs=10)), type='supervised'), user_provided_suffix='fine-tuned', metadata=None, usage_metrics=None, shared_with_openai=False, eval_id=None)

In [27]:
from IPython.display import clear_output
import time

break_loop = False

while True:
    
    clear_output(wait=True)  # clear output on each iteration

    events = openai.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id, limit=10).data

    for event in events:
        print(f"Event ID: {event.id}")
        print(f"Created At: {event.created_at}")
        print(f"Level: {event.level}")
        print(f"Message: {event.message}")
        print(f"Data: {event.data}")
        print("------")
        
        # flags to exit the loop
        if "The job has successfully completed" in event.message or event.level == "error":
            break_loop = True

    if break_loop:
        break
    
    time.sleep(20)  # wait 20 seconds on each iteration

Event ID: ftevent-QJQaFrBoTgeSU8uhv97UURMF
Created At: 1747403589
Level: info
Message: The job has successfully completed
Data: {}
------
Event ID: ftevent-812iJTX7LrIeX2QdmD0k1lwz
Created At: 1747403583
Level: info
Message: New fine-tuned model created
Data: {}
------
Event ID: ftevent-671RVL7d8aN9S0UXhA1QyDCV
Created At: 1747403583
Level: info
Message: Checkpoint created at step 1530
Data: {}
------
Event ID: ftevent-JP0QaChSa597iJr21uB3yZjr
Created At: 1747403583
Level: info
Message: Checkpoint created at step 1360
Data: {}
------
Event ID: ftevent-3HN2zO3aKjrxNAR5VJL8jfNp
Created At: 1747403556
Level: info
Message: Step 1700/1700: training loss=0.00
Data: {'step': 1700, 'train_loss': 4.695012103184126e-05, 'total_steps': 1700, 'train_mean_token_accuracy': 1.0}
------
Event ID: ftevent-z7VNdocgyYMGv3Mpxvk6DgOR
Created At: 1747403556
Level: info
Message: Step 1699/1700: training loss=0.00
Data: {'step': 1699, 'train_loss': 4.38690185546875e-05, 'total_steps': 1700, 'train_mean_token_

In [30]:
model_name = openai.fine_tuning.jobs.retrieve(job_id).fine_tuned_model
print(fine_tuned_model_name)

ft:gpt-4o-mini-2024-07-18:personal:fine-tuned:BXpmhXeF


In [31]:
# TEST AFTER FINE TUNING

ask_for_test_prompts(True)

using ft:gpt-4o-mini-2024-07-18:personal:fine-tuned:BXpmhXeF model

question: what is the temperature in Tegucigalpa today?

  -answer: It's a cool 22°C in Tegucigalpa today.

question: what is the temperature in Yamoussoukro today?

  -answer: It's a warm 30°C in Yamoussoukro today.

question: what is the temperature in Rabat today?

  -answer: It's a cool 18°C in Rabat today.

question: what is the temperature in Wellington today?

  -answer: It's a cool 12°C in Wellington today.

question: what is the temperature in New York today?

  -answer: It's a cool 58°F in New York today.

